<a href="https://colab.research.google.com/github/nunocesarsa/GoogleColab_public/blob/master/DSRP_Lunch_KFolding_ML_Simulated_inversion_V02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
#loading up my google drive
from google.colab import drive
drive.mount('/content/drive')
#Perhaps this step can be skipped by saving directly to the workspace


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading up all the packages

In [130]:
#package instalation

#Installing PROSAIL
!pip install prosail

#latin hypercube stuff
#lets try to do a LHS
!pip install lhsmdu

#this package as a number of functions to deal with hyperspectral data
!pip install pysptools


In [0]:
#General purpose: 
import matplotlib.pyplot as plt
import numpy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

#the beutiful R like data frame
import pandas as pd

#PROSPECT+SAIL Radiative transfer mode package
import prosail

#Sampling design package
import lhsmdu

#a few more stuff for random
import random as rdm
import math


#package to for operations on spectral data
import pysptools as sptool 
from pysptools import distance


from sklearn.model_selection import KFold # import KFold

#machine learning stuff
#NEURAL NETWORK - Keras will be updated soon so this colab will also have to be changed

from sklearn.neural_network import MLPRegressor as ANN_reg #this is a simpler neural network package
from keras.models import Sequential, Model
from keras.layers import Dense, Input
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#ignore the warning for now

#Random FOREST
# Fitting Random Forest Regression to the dataset 
# import the regressor 
from sklearn.ensemble import RandomForestRegressor 

#Gaussian processes
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel, ExpSineSquared, RationalQuadratic, RBF

#aux functions
from sklearn.preprocessing import MinMaxScaler #this is to standardize the input data [not used for now]
from sklearn import metrics

#for model storing -sklearn
import pickle

#for model storing keras
from keras.models import model_from_json


# Custom functions to facilitate the calling of the prosail and convolution of the spectral wavelength to sentinel data

In [0]:
def custom_prosail(cab,cw,cm,lai):
  import prosail
  #default parameters
  n= 1.
  car=10.
  cbrown=0.01
  typelidf=1 #this is the default option
  lidfa = -1 #leaf angle distribution parameter a and b
  lidfb=-0.15
  hspot= 0.01 #hotspot parameters - got this from R package https://www.rdocumentation.org/packages/hsdar/versions/0.4.1/topics/PROSAIL
  #sun and viewing angle
  tts=30. #observation and solar position parameters
  tto=10. 
  psi=0.
  #for now i put them by hand but they should be an input of a custom function
  #tts=sol_zen #solar zenith angle
  #tto=inc_zen #sensor zenith angle
  #psi=raa
  rho_out = prosail.run_prosail(n,
                                 cab,
                                 car,
                                 cbrown,
                                 cw,
                                 cm,
                                 lai,
                                 lidfa,
                                 hspot,
                                 tts,tto,psi,
                                 typelidf, lidfb,
                                 prospect_version="D",
                                 factor='SDR', 
                                 rsoil=.5, psoil=.5)
  return(rho_out)

In [0]:
def Prosail2S2(path2csv,spectra_input):
  #importing pandas
  import pandas as pd
  import numpy
  import numpy as np
  #upload a S2_Response.csv from https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

  s2_table = pd.read_csv(path2csv,sep=";",decimal=",") #check if this is proper, regarding the sep and dec
  #chekc which row you are actually extracting

  s2_table_sel = s2_table[s2_table['SR_WL'].between(400,2500)] #selects all values between 400 and 2500
  spectra_input_df = pd.DataFrame(data=spectra_input,columns=["rho"],index=s2_table_sel.index) #transforms the input array into a pandas df with the column name rho and row.index = to the original input table

  
  rho_s2 = s2_table_sel.multiply(spectra_input_df['rho'],axis="index") #calculates the numerator
  w_band_sum = s2_table_sel.sum(axis=0,skipna = True) #calculates the denominator

  output = (rho_s2.sum(axis=0)/w_band_sum).rename_axis("ID").values #runs the weighted mean and converts the output to a numpy array

  return output[1:] #removes the first value because it represents the wavelength column

#please LOAD THTE FILE NOW
filepath="/content/drive/My Drive/S2_Response.csv"
#filepath="/content/S2_Responses_S2B.csv"
#filepath="/content/drive/My Drive/S2_Response.csv"



In [0]:
def Gen_spectra_data(traits):
  k = 1
  #pd_train_traits=traits
  #print(range(len(traits)))
  for i in range(len(traits)):
    #n_t = pd_train_traits["n"][i]
    cab_t = traits["cab"][i]
    #car_t = pd_train_traits["car"][i]
    #cbrown_t = pd_train_traits["cbrown"][i]
    cw_t = traits["cw"][i]
    cm_t = traits["cm"][i]
    lai_t = traits["lai"][i]

    if k == 1:
      tr_rho_s = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_s = Prosail2S2(filepath,tr_rho_s)
      #plt.plot ( x, tr_rho_s, ':', label="Training prosail")
      #plt.legend(loc='best')
      
    if k > 1:
      tr_rho_t = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_t = Prosail2S2(filepath,tr_rho_t)
      tr_rho_s = np.vstack((tr_rho_s,tr_rho_t))
      #plt.plot ( x, tr_rho_t, ':')

    k = k+1


  rho_samples=tr_rho_s


  return rho_samples

In [0]:
#function to store the model outputs onto a dictionary

column_names=["Model",
              "NSamples",
              "OutOfBag",
              "KFold_tr_samples",
              "KFold_vl_samples",
              "Variable",
              "Fold_nr",
              "ExplVar",
              "Max_err",
              "Mean_abs_Err",
              "Mean_sqr_err",
              #"Mean_sqr_lg_err",
              "Median_abs_err",
              "r2",
              "MAPE",
              "ModelName"]
              #"Mean_poiss_dev",
              #"Mean_gamma_dev"]
              #"Mean_tweed_dev"]

#mape is not existant in the package so we have to create it:
#https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn
#from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  #Here, the file is used for saving
  #creating a df to receive the data


def calc_metrics(MDL,Samples,oob_samples,kf_tr,kf_vl,Variable,Fold,Ref,Pred,Modelname):

  out_list = {"Model":MDL,
              "NSamples":Samples,
              "OutOfBag":oob_samples,
              "KFold_tr_samples":kf_tr,
              "KFold_vl_samples":kf_vl,
              "Variable":Variable,
              "Fold_nr":Fold,
              "ExplVar": metrics.explained_variance_score(Ref, Pred),
              "Max_err": metrics.max_error(Ref, Pred),
              "Mean_abs_Err": metrics.mean_absolute_error(Ref, Pred),
              "Mean_sqr_err": metrics.mean_squared_error(Ref, Pred),
              "Median_abs_err" : metrics.median_absolute_error(Ref, Pred),
              "r2": metrics.r2_score(Ref, Pred),
              "MAPE": mean_absolute_percentage_error(Ref, Pred),
              "ModelName":Modelname}


  return out_list






#Important section Generating a trait space

In [0]:
#number of samples
#train_n3000 = 3000
#train_n1500 = 1500
#train_n0500 = 500

n_traits=4 #I will test on 4 varying traits: cab, car, cw,cm,lai

#generating a LHS hypercube (it uses a 0 to 1 interval that can be used as a multiplier against the different traits)
np.random.seed(0)
#LHS_train3000 = lhsmdu.createRandomStandardUniformMatrix(n_traits,train_n3000 ) #the package has a more advanced method but it is too slow to process
#LHS_train1500 = lhsmdu.createRandomStandardUniformMatrix(n_traits,train_n1500 )
#LHS_train0500 = lhsmdu.createRandomStandardUniformMatrix(n_traits,train_n0500 )

#max_n=1 #this value should go from 1 to 2, so i make it change from 0 to 1 here and then add 1 later
max_cab=79. #add 1
#max_car=44. #add 1
#max_cbrown= 9.99 #add 0.01
max_cw=0.008 #add 0.001 
max_cm=0.008 #0.001
max_lai = 9.9 #add 0.1

min_cab = 10.
min_cw = 0.005
min_cm = 0.005
min_lai = .5



Setting up the models

In [0]:
#Random forest
#initializing the random forest
rfr_ml = RandomForestRegressor(n_estimators=1000,random_state=0,
                               min_samples_leaf=5,
                               min_samples_split=5,
                               verbose=0)

#single task Artificial neural network
ann_ml = Sequential()
#ann_ml.add(Dense(9, input_dim=9, activation='linear'))
ann_ml.add(Dense(10, input_dim=9, activation='tanh'))
ann_ml.add(Dense(6, activation='relu'))
ann_ml.add(Dense(4)) #indeed this ha to be added in this case without any activ function, the R script added this on its own
#model.add(Dense(1, activation='sigmoid'))

# compile the keras model
ann_ml.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

#gaussian processes with default acquisition function
#initializing the gaussian process
gpr_ml = GaussianProcessRegressor(n_restarts_optimizer=50,
                                  normalize_y=True,
                                  random_state=0)

#multi-task neural network - 32 by 16 by 8 per task
inputs = Input(shape=(9,))
sub1 = Dense(32, activation='tanh')(inputs)
sub2 = Dense(16, activation='tanh')(sub1)
cab1 = Dense(8, activation='sigmoid')(sub2)
cw1  = Dense(8, activation='sigmoid')(sub2)
cm1  = Dense(8, activation='sigmoid')(sub2)
lai1 = Dense(8, activation='sigmoid')(sub2)
cab2 = Dense(1, activation='linear')(cab1)
cw2  = Dense(1, activation='linear')(cw1)
cm2  = Dense(1, activation='linear')(cm1)
lai2 = Dense(1, activation='linear')(lai1)


#building the model using keras api
model = Model(inputs=inputs, outputs=[cab2,cw2,cm2,lai2])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])



# Main function Loop

This is a confusing loop at first sight.

First it generates a number of samples (j) based on input, then k-folds the data and runs all models storing their accuraccy results. Then stores all data into a list so i can operate on it elsewhere. 

In [144]:
#setting up the limits of the run

#minimium samples
n_samples = 200
n_points = np.arange(n_samples,3000+1,200)
print("Loop for the following set of LHS samples:",n_points)


Loop for the following set of LHS samples: [ 200  400  600  800 1000 1200 1400 1600 1800 2000 2200 2400 2600 2800
 3000]


In [0]:
### storing the dataset

#this is for the training dataset
df_metrics = pd.DataFrame(columns=column_names)
#this is for the out of bag
df_metrics_valid = pd.DataFrame(columns=column_names)

#a path to store the models 
path2folder = "/content/drive/My Drive/DSRP_Lunch_outputs/Models/"

#minimium samples
#n_samples = 500
#n_points = np.arange(n_samples,6000+1,500)

print("Loop for the following set of LHS samples:",n_points)

#this command is enough to set u the k-fold
fold_nr=5
kf = KFold(n_splits=fold_nr,shuffle=True,random_state=0) # Define the split 

#this next comment snippet shouldn't be needed anymore

##here we generate the out of bag test dataset of 1000 points that is used as out-of-bag testing
#LHS_valid = lhsmdu.createRandomStandardUniformMatrix(n_traits,1000)
#valid_pd_trait = pd.DataFrame.transpose(pd.DataFrame(LHS_valid))
#valid_pd_trait.columns = ["cab","cw","cm","lai"]

#valid_pd_trait["cab"]=valid_pd_trait["cab"]*max_cab+min_cab
#valid_pd_trait["cw"] =valid_pd_trait["cw"] *max_cw+min_cw
#valid_pd_trait["cm"] =valid_pd_trait["cm"] *max_cm+min_cw
#valid_pd_trait["lai"]=valid_pd_trait["lai"]*max_lai+min_lai

#valid_np_trait = valid_pd_trait.iloc[:,:].values
#valid_np_spect = Gen_spectra_data(valid_pd_trait)
#valid_df_spect = valid_np_spect[:,[1,2,3,4,5,6,8,11,12]]

for j in n_points:
  print("Generating trait table for",j,"samples")

  #this section generates trait space - careful with the column names as this is significant later
  LHS_train = lhsmdu.createRandomStandardUniformMatrix(n_traits,j)
  pd_trait = pd.DataFrame.transpose(pd.DataFrame(LHS_train))
  pd_trait.columns = ["cab","cw","cm","lai"]

  pd_trait["cab"]=pd_trait["cab"]*max_cab+min_cab
  pd_trait["cw"] =pd_trait["cw"] *max_cw+min_cw
  pd_trait["cm"] =pd_trait["cm"] *max_cm+min_cw
  pd_trait["lai"]=pd_trait["lai"]*max_lai+min_lai

  #lets also create a numpy object for the traits - this is for ease of operation later
  np_trait = pd_trait.iloc[:,:].values

  #until here its instantaneous

  #now we first generate the date in hyperspectral, convolute it to S2 while iterating through the entire given trait space
  np_spect = Gen_spectra_data(pd_trait)[:,[1,2,3,4,5,6,8,11,12]]
  #trait_df = np_spectra

  #now from these generated samples, we take out 10% of the points (aproximately, sometimes its a decimal more, depends on the number)
  #creating row identifiers
  #this gets the length for the entire list
  index = list(range(len(np_spect)))

  index10 = rdm.sample(index,math.ceil(len(index)*.1)) #randomly selects 10% of te data (aproximately)
  index90 = [x for x in index if x not in index10] #makes a list set wit the reamining
  #once we have the list of point, we can subset

  vl_trait_df = np_trait[index10,]
  tr_trait_df = np_trait[index90,]
  
  vl_spect_df = np_spect[index10,]
  tr_spect_df = np_spect[index90,]

  #this is a function to re-scale inputs for the neural networks
  scaler = MinMaxScaler()

  print("Of which, 10% were taken as out-of-bag:", len(vl_trait_df))

  #now we have to iterate per fold, train and store the prediction
  k=1
  for train_index, test_index in kf.split(tr_spect_df):

    #print("Calculating fold ",k,"of",fold_nr)
    X_train, X_test = tr_spect_df[train_index], tr_spect_df[test_index]
    Y_train, Y_test = tr_trait_df[train_index], tr_trait_df[test_index]
    label_names = ["cab","cw","cm","lai"]

    #training the machine learning models
    #random forest
    rfr_ml.fit(X_train,Y_train)
    #shallow multi objective single task neural network
    scaler.fit(Y_train)
    Y_train_norm = scaler.transform(Y_train)
    ann_ml.fit(X_train,Y_train_norm,epochs=500,verbose=0)

    #GPR - Training 
    gpr_ml.fit(X_train,Y_train)

    #"Shallow" Multi task ANN training
    cab_train = Y_train_norm[:,0]
    cw_train = Y_train_norm[:,1]
    cm_train = Y_train_norm[:,2]
    lai_train = Y_train_norm[:,3]
    model.fit(X_train,[cab_train,cw_train,cm_train,lai_train],epochs=250,verbose=0)#hides spam on the output


    #Prediction on training fold
    y_ann = scaler.inverse_transform(ann_ml.predict(X_test))
    y_rfr = rfr_ml.predict(X_test)
    y_gpr = gpr_ml.predict(X_test)

    #the multi task is a bit more complicated
    Y_mtn = model.predict(X_test)
    cab_pred = Y_mtn[0]
    cw_pred  = Y_mtn[1]
    cm_pred  = Y_mtn[2]
    lai_pred = Y_mtn[3]
    np_Y_ann_pred = np.array(cab_pred)
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,cw_pred))
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,cm_pred))
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,lai_pred))

    y_mtn = scaler.inverse_transform(np_Y_ann_pred)

    #prediction on out of bag
    y_ann_valid = scaler.inverse_transform(ann_ml.predict(vl_spect_df))
    y_rfr_valid = rfr_ml.predict(vl_spect_df)
    y_gpr_valid = gpr_ml.predict(vl_spect_df)

    Y_mtn_valid = model.predict(vl_spect_df)
    cab_pred_valid = Y_mtn_valid[0]
    cw_pred_valid  = Y_mtn_valid[1]
    cm_pred_valid  = Y_mtn_valid[2]
    lai_pred_valid = Y_mtn_valid[3]
    np_Y_ann_pred_valid = np.array(cab_pred_valid)
    np_Y_ann_pred_valid = np.hstack((np_Y_ann_pred_valid,cw_pred_valid))
    np_Y_ann_pred_valid = np.hstack((np_Y_ann_pred_valid,cm_pred_valid))
    np_Y_ann_pred_valid = np.hstack((np_Y_ann_pred_valid,lai_pred_valid))

    y_mtn_valid = scaler.inverse_transform(np_Y_ann_pred_valid)


    #setting the model filenames
    #zfill is a handy function to standardize the output, i love it!
    rfr_ml_modelname = path2folder+"RFR"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".pkl"
    gpr_ml_modelname = path2folder+"GPR"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".pkl"
    ann_ml_modelname = path2folder+"ANN"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".h5"
    mtn_ml_modelname = path2folder+"MTN"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".h5"

    #COMMENT HERE IF YOU DO NOT WANT TO SAVE THE MODELS
    #saving the models
    #sklearn are pickles while keras are h5 files
    #not saving random forest for now since i am pretty sure it wont be good but it is a big file...
    #with open(rfr_ml_modelname, 'wb') as file:
    #  pickle.dump(rfr_ml, file)

    #with open(gpr_ml_modelname, 'wb') as file:
    #  pickle.dump(gpr_ml, file)

    #ann_ml.save(ann_ml_modelname)
    #model.save(mtn_ml_modelname)
      


    #the next section stores all the results
    for i in range(n_traits):

      #this stores the training accuraccy metrics
      #print(i)
      ann_temp_list = calc_metrics("ANN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_ann[:,i],ann_ml_modelname)
      rfr_temp_list = calc_metrics("RFR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_rfr[:,i],rfr_ml_modelname)
      gpr_temp_list = calc_metrics("GPR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_gpr[:,i],gpr_ml_modelname)
      mtn_temp_list = calc_metrics("MTN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_mtn[:,i],mtn_ml_modelname)

      #This stores the metrics against the out of bag list
      ann_temp_list_valid = calc_metrics("ANN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_ann_valid[:,i],ann_ml_modelname)
      rfr_temp_list_valid = calc_metrics("RFR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_rfr_valid[:,i],rfr_ml_modelname)
      gpr_temp_list_valid = calc_metrics("GPR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_gpr_valid[:,i],gpr_ml_modelname)
      mtn_temp_list_valid = calc_metrics("MTN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_mtn_valid[:,i],mtn_ml_modelname)

      #appending to the dataframe (training error)
      df_metrics = df_metrics.append(ann_temp_list,ignore_index=True)
      df_metrics = df_metrics.append(rfr_temp_list,ignore_index=True)
      df_metrics = df_metrics.append(gpr_temp_list,ignore_index=True)
      df_metrics = df_metrics.append(mtn_temp_list,ignore_index=True)

      #appending to the dataframe (out of bag error)
      df_metrics_valid = df_metrics_valid.append(ann_temp_list_valid,ignore_index=True)
      df_metrics_valid = df_metrics_valid.append(rfr_temp_list_valid,ignore_index=True)
      df_metrics_valid = df_metrics_valid.append(gpr_temp_list_valid,ignore_index=True)
      df_metrics_valid = df_metrics_valid.append(mtn_temp_list_valid,ignore_index=True)
    

    k=k+1

    #saves a temp version of the files
    #saving the file to an output
  
  print("Of which",len(X_train),"were used for training and",len(X_test),"for validation")
  df_metrics.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/Temp_KFold_TrainingError_csv2.csv",sep=";",decimal=",")
  df_metrics_valid.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/Temp_KFold_OutOfBag_csv2.csv",sep=";",decimal=",")

  df_metrics.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/Temp_KFold_TrainingError_csv1.csv",sep=",",decimal=".")
  df_metrics_valid.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/Temp_KFold_OutOfBag_csv1.csv",sep=",",decimal=".")


#saving the file to an output
df_metrics.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/KFold_TrainingError_csv2.csv",sep=";",decimal=",")
df_metrics_valid.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/KFold_OutOfBag_csv2.csv",sep=";",decimal=",")

df_metrics.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/KFold_TrainingError_csv1.csv",sep=",",decimal=".")
df_metrics_valid.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/KFold_OutOfBag_csv1.csv",sep=",",decimal=".")


Loop for the following set of LHS samples: [ 200  400  600  800 1000 1200 1400 1600 1800 2000 2200 2400 2600 2800
 3000]
Generating trait table for 200 samples
Of which, 10% were taken as out-of-bag: 20
Of which 144 were used for training and 36 for validation
Generating trait table for 400 samples
Of which, 10% were taken as out-of-bag: 40
Of which 288 were used for training and 72 for validation
Generating trait table for 600 samples
Of which, 10% were taken as out-of-bag: 60
Of which 432 were used for training and 108 for validation
Generating trait table for 800 samples
Of which, 10% were taken as out-of-bag: 80
Of which 576 were used for training and 144 for validation
Generating trait table for 1000 samples
Of which, 10% were taken as out-of-bag: 100
Of which 720 were used for training and 180 for validation
Generating trait table for 1200 samples
Of which, 10% were taken as out-of-bag: 120
Of which 864 were used for training and 216 for validation
Generating trait table for 1400

# Plotting stuff

For now in R 